* user_action.csv：用户行为表
* feed_info.csv：Feed信息表（其中feed表示的是内容流，也就是content feed，一系列按照特定顺序或者基于某种规则展示給用户的内容集合）
* feed_embeddings.csv：Feed向量表
* test_a.csv：A榜测试集
* submit_demo_初赛a.csv：A榜提交结果demo

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import json
import utils
import importlib
importlib.reload(utils)
from utils import *

## 数据读取

In [57]:
feed_df=pd.read_csv('./data/feed_info.csv')
feed_df.head(3)

,feedid,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,43549,6165,38,104741 122649 8109 117252 65632 23463 118668 4...,139499 59421 82007 142955 27736 83577 52394 11...,142955 27736 83577 103956 32010 34170 89740 90...,19356.0,11703.0,15506;7715;17582,26334;219;25209;7715;18541,81;269;159;6,269 0.8525666;81 0.8525666;8 1.1e-07;306 0.0;2...,26439 5247 6426 3827 1882 26018 20744 22204 30...,25926 8491 13394 2203 26439 6243 33054 16435 1...,2203 26439 6243 33054 16435 16307 17070 24908 ...
1,77432,9386,60,35753 27736 146603 73055 11794 101761 11794 81...,35753 146603 73055 11794 101761 67496 16933 52...,146739 14368 79290 79213 47366 8109 33194 1198...,NaN,NaN,8199;18322;4243,24078;19924,194;267;159;6,267 0.99293476;194 0.99293476,31010 32495 6243 13923 15360 30483 2709 26084 ...,31010 32495 13923 15360 30483 2709 26084 15160...,7259 20851 5061 26207 17573 17531 15117 20072 ...
2,12921,2996,15,114413 107973 117252 27736 41035 32715 125374 ...,NaN,NaN,10288.0,11354.0,NaN,11696;6926,202;23;160;6,23 0.64771646;202 0.64771646,27077 10050 27464 20072 26018 6243 21492 14750...,NaN,NaN


In [58]:
action_df=pd.read_csv('./data/user_action.csv')
action_df.head(3)

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite
0,8,71474,1,1,0,0,1,500,5366,0,0,0,0
1,8,73916,1,1,0,0,0,250,1533,0,0,0,0
2,8,50282,1,1,0,0,0,750,1302,0,0,0,0


In [59]:
# 预测数据
test=pd.read_csv('./data/test_a.csv')
test.head(3)

,userid,feedid,device,read_comment,like,click_avatar,forward
0,14298,67227,1,NaN,NaN,NaN,NaN
1,68356,91864,2,NaN,NaN,NaN,NaN
2,49925,104657,2,NaN,NaN,NaN,NaN


In [60]:
# 合并数据
merged_df = pd.merge(action_df, feed_df, on='feedid', how='left')
merged_df.head(3)

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,8,71474,1,1,0,0,1,500,5366,0,0,0,0,1528,11,110142 37527 123078 110142 134872 294 146821 2...,50275 5643 42751 120238 83167 1371 93533 89031...,104002 104002 104002 104002 104002 104002 1040...,13745.0,3556.0,NaN,7978;9680,219;6;124,266 0.68073416;41 0.68073416,24513 8512 27464 24513 30027 73 23103 6243 287...,11458 1254 28079 20753 23952 10212 8740 32214 ...,23112 23112 23112 23112 23112 23112 23112 2311...
1,8,73916,1,1,0,0,0,250,1533,0,0,0,0,1442,16,NaN,NaN,117252 44399 22662 4438,NaN,NaN,NaN,4388;24079;19399;19166;22371;5615,308;6,266 0.39245349;261 0.39245349;8 3.8e-07;306 0....,NaN,NaN,26018 10043 30420 969
2,8,50282,1,1,0,0,0,750,1302,0,0,0,0,8648,31,95647 49085 1824 140316 56344 45723 121036 977...,NaN,104002 104002 104002 104002 104002 104002 1040...,NaN,NaN,NaN,5077;17881;5132;25008;61;17812,9;6;222;222,16 0.33405718;332 0.33405718;8 7e-08;306 0.0;2...,21277 21277 1873 3110 32556 22468 20201 12785 ...,NaN,23112 23112 23112 23112 23112 23112 23112 2311...


### 数据基本信息查看

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7739867 entries, 0 to 7739866
Data columns (total 27 columns):
 #   Column                Dtype  
---  ------                -----  
 0   userid                int64  
 1   feedid                int64  
 2   date_                 int64  
 3   device                int64  
 4   read_comment          float64
 5   comment               float64
 6   like                  float64
 7   play                  float64
 8   stay                  float64
 9   click_avatar          float64
 10  forward               float64
 11  follow                float64
 12  favorite              float64
 13  authorid              int64  
 14  videoplayseconds      int64  
 15  description           object 
 16  ocr                   object 
 17  asr                   object 
 18  bgm_song_id           float64
 19  bgm_singer_id         float64
 20  manual_keyword_list   object 
 21  machine_keyword_list  object 
 22  manual_tag_list       object 
 23  machine

In [52]:
# 分析prediction_df数据集
num_preds, cat_preds = analyze_data(merged_df, "数据集")


分析数据集: 数据集
总行数: 7317882
总列数: 27

前5行数据:


,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,...,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,8,71474,1,1,0,0,1,500,5366,0,...,104002 104002 104002 104002 104002 104002 1040...,13745.0,3556.0,NaN,7978;9680,219;6;124,266 0.68073416;41 0.68073416,24513 8512 27464 24513 30027 73 23103 6243 287...,11458 1254 28079 20753 23952 10212 8740 32214 ...,23112 23112 23112 23112 23112 23112 23112 2311...
1,8,73916,1,1,0,0,0,250,1533,0,...,117252 44399 22662 4438,NaN,NaN,NaN,4388;24079;19399;19166;22371;5615,308;6,266 0.39245349;261 0.39245349;8 3.8e-07;306 0....,NaN,NaN,26018 10043 30420 969
2,8,50282,1,1,0,0,0,750,1302,0,...,104002 104002 104002 104002 104002 104002 1040...,NaN,NaN,NaN,5077;17881;5132;25008;61;17812,9;6;222;222,16 0.33405718;332 0.33405718;8 7e-08;306 0.0;2...,21277 21277 1873 3110 32556 22468 20201 12785 ...,NaN,23112 23112 23112 23112 23112 23112 23112 2311...
3,8,11391,1,1,0,0,1,3750,5191,0,...,142955 80924 45012 25794 16650 8097,13096.0,5012.0,24390;14546;6355,17918;131;22816,228;12;159;6,12 0.867993;228 0.867993;8 6.3e-07;306 0.0;207...,7357 22439 31380 12371 18881 26018 2223 10045 ...,6244 28921 12837 199 30086,2203 26439 17681 17681 10226 8481 3703 1872
4,8,27349,1,1,0,0,0,250,800,0,...,150623 17721 287 66591 4438,22215.0,7899.0,9868;27100;11411,23257;20650;23257;8693;26486;25995;7144;19699,322;13;159;6,266 0.78305119;41 0.78305119;8 0.0;306 0.0;207...,17297 28718 10155 5967 17297 14126 26193 20860...,13519 14749 12 19791 20857 14040 22468 89 8481...,5990 15640 30749 29499 66 4956 7578 969



数据类型统计:
int64      15
object     10
float64     2
Name: count, dtype: int64

数值型特征(numeric_features):
['date_', 'device', 'read_comment', 'comment', 'like', 'click_avatar', 'forward', 'follow', 'favorite', 'videoplayseconds']

类别型特征(categorical_features):
['userid', 'feedid', 'play', 'stay', 'authorid', 'description', 'ocr', 'asr', 'bgm_song_id', 'bgm_singer_id', 'manual_keyword_list', 'machine_keyword_list', 'manual_tag_list', 'machine_tag_list', 'description_char', 'ocr_char', 'asr_char']

数值型特征描述统计:


,count,mean,std,min,25%,50%,75%,max
date_,7317882.0,7.801455,4.063833,1.0,4.0,8.0,11.0,14.0
device,7317882.0,1.765396,0.423751,1.0,2.0,2.0,2.0,2.0
read_comment,7317882.0,0.035016,0.183820,0.0,0.0,0.0,0.0,1.0
comment,7317882.0,0.000405,0.020111,0.0,0.0,0.0,0.0,1.0
like,7317882.0,0.025805,0.158553,0.0,0.0,0.0,0.0,1.0
click_avatar,7317882.0,0.007533,0.086467,0.0,0.0,0.0,0.0,1.0
forward,7317882.0,0.003821,0.061698,0.0,0.0,0.0,0.0,1.0
follow,7317882.0,0.000721,0.026844,0.0,0.0,0.0,0.0,1.0
favorite,7317882.0,0.001342,0.036615,0.0,0.0,0.0,0.0,1.0
videoplayseconds,7317882.0,32.255837,109.232491,2.0,14.0,25.0,53.0,25200.0



类别型特征唯一值统计:

userid:
唯一值数量: 20000
前10个最常见值:
userid
32579     980
9541      974
69134     969
239494    968
246409    964
70684     962
39554     962
114022    961
118651    960
100226    958
Name: count, dtype: int64

feedid:
唯一值数量: 96564
前10个最常见值:
feedid
46022     6841
88646     4442
101297    4419
24381     4310
73903     4304
64025     4255
67305     4077
100591    3889
100065    3821
5662      3745
Name: count, dtype: int64

play:
唯一值数量: 201721
前10个最常见值:
play
0       986828
750      47166
500      28029
1250     27896
1000     26532
1500     26298
2000     22883
1750     22057
2500     19571
2250     17332
Name: count, dtype: int64

stay:
唯一值数量: 220343
前10个最常见值:
stay
0      1077
998     806
996     775
1       772
999     757
997     735
975     725
977     724
976     720
978     719
Name: count, dtype: int64

authorid:
唯一值数量: 18429
前10个最常见值:
authorid
18286    32956
14028    27841
14653    25149
4537     23040
11296    22529
1072     21457
15610    20949
12577    19760
2979     1

bgm_singer_id           3674581
bgm_song_id             3674581
manual_keyword_list     2255381
ocr_char                1905399
ocr                     1886890
asr_char                1721179
asr                     1721179
machine_keyword_list     848552
description              219807
description_char         217989
machine_tag_list          28759
manual_tag_list            6707
dtype: int64

In [53]:
num_feeds, cat_feeds = improved_analyze_data(merged_df, "合并数据集")


高级分析数据集: 合并数据集

最终数值型特征:
['comment', 'favorite', 'read_comment', 'click_avatar', 'userid', 'like', 'forward', 'feedid', 'stay', 'play', 'follow', 'authorid', 'date_', 'videoplayseconds']

最终类别型特征:
['bgm_song_id', 'ocr_char', 'userid', 'manual_tag_list', 'description', 'bgm_singer_id', 'asr_char', 'device', 'description_char', 'asr', 'manual_keyword_list', 'feedid', 'machine_tag_list', 'authorid', 'machine_keyword_list', 'ocr']

文本特征需要特殊处理:
['description', 'ocr', 'asr', 'manual_keyword_list', 'machine_keyword_list', 'manual_tag_list', 'machine_tag_list']

高缺失值特征:
['ocr', 'asr', 'bgm_song_id', 'bgm_singer_id', 'manual_keyword_list', 'machine_keyword_list', 'ocr_char', 'asr_char']


In [54]:
pd.set_option('display.max_columns', None)
merged_df.head()

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,8,71474,1,1,0,0,1,500,5366,0,0,0,0,1528,11,110142 37527 123078 110142 134872 294 146821 2...,50275 5643 42751 120238 83167 1371 93533 89031...,104002 104002 104002 104002 104002 104002 1040...,13745.0,3556.0,NaN,7978;9680,219;6;124,266 0.68073416;41 0.68073416,24513 8512 27464 24513 30027 73 23103 6243 287...,11458 1254 28079 20753 23952 10212 8740 32214 ...,23112 23112 23112 23112 23112 23112 23112 2311...
1,8,73916,1,1,0,0,0,250,1533,0,0,0,0,1442,16,NaN,NaN,117252 44399 22662 4438,NaN,NaN,NaN,4388;24079;19399;19166;22371;5615,308;6,266 0.39245349;261 0.39245349;8 3.8e-07;306 0....,NaN,NaN,26018 10043 30420 969
2,8,50282,1,1,0,0,0,750,1302,0,0,0,0,8648,31,95647 49085 1824 140316 56344 45723 121036 977...,NaN,104002 104002 104002 104002 104002 104002 1040...,NaN,NaN,NaN,5077;17881;5132;25008;61;17812,9;6;222;222,16 0.33405718;332 0.33405718;8 7e-08;306 0.0;2...,21277 21277 1873 3110 32556 22468 20201 12785 ...,NaN,23112 23112 23112 23112 23112 23112 23112 2311...
3,8,11391,1,1,0,0,1,3750,5191,0,0,0,0,11976,6,32715 49575 54606 85056 90843 23031 61533 9356...,27737 129801 37630 755 95811 95811,142955 80924 45012 25794 16650 8097,13096.0,5012.0,24390;14546;6355,17918;131;22816,228;12;159;6,12 0.867993;228 0.867993;8 6.3e-07;306 0.0;207...,7357 22439 31380 12371 18881 26018 2223 10045 ...,6244 28921 12837 199 30086,2203 26439 17681 17681 10226 8481 3703 1872
4,8,27349,1,1,0,0,0,250,800,0,0,0,0,4370,12,117910 44854 26564 51353 117763 57127 134820 1...,3195 65625 79 89307 93563 62197 127203 25794 2...,150623 17721 287 66591 4438,22215.0,7899.0,9868;27100;11411,23257;20650;23257;8693;26486;25995;7144;19699,322;13;159;6,266 0.78305119;41 0.78305119;8 0.0;306 0.0;207...,17297 28718 10155 5967 17297 14126 26193 20860...,13519 14749 12 19791 20857 14040 22468 89 8481...,5990 15640 30749 29499 66 4956 7578 969


### 数据预处理

In [61]:
import os
import copy
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [62]:
# 合并数据
test['date_'] = 15
action_df = pd.concat([action_df,test])

In [63]:
# 标签列
target = ["read_comment",  # 是否查看评论
          "like",  # 是否点赞
          "click_avatar",  # 是否点击头像
          "forward"] # 是否转发
# 稀疏特征
sparse_features = ['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id']
# 变长序列特征
varlen_features = ['manual_tag_list', # 人工标注的分类标签
                   'manual_keyword_list', 'description'] # 机器标注的关键词
# 稠密特征
dense_features = ['videoplayseconds']

In [64]:
# 数据合并
feed_df = feed_df[['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id','manual_tag_list','manual_keyword_list', 'description']]
data = action_df.merge(feed_df, how='left',on='feedid') #行为数据拼接，作者id，bgm_song_id 

In [65]:
pd.set_option('display.max_columns',None)
data.head()

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,authorid,videoplayseconds,bgm_song_id,bgm_singer_id,manual_tag_list,manual_keyword_list,description
0,8,71474,1,1,0.0,0.0,1.0,500.0,5366.0,0.0,0.0,0.0,0.0,1528,11,13745.0,3556.0,219;6;124,NaN,110142 37527 123078 110142 134872 294 146821 2...
1,8,73916,1,1,0.0,0.0,0.0,250.0,1533.0,0.0,0.0,0.0,0.0,1442,16,NaN,NaN,308;6,NaN,NaN
2,8,50282,1,1,0.0,0.0,0.0,750.0,1302.0,0.0,0.0,0.0,0.0,8648,31,NaN,NaN,9;6;222;222,NaN,95647 49085 1824 140316 56344 45723 121036 977...
3,8,11391,1,1,0.0,0.0,1.0,3750.0,5191.0,0.0,0.0,0.0,0.0,11976,6,13096.0,5012.0,228;12;159;6,24390;14546;6355,32715 49575 54606 85056 90843 23031 61533 9356...
4,8,27349,1,1,0.0,0.0,0.0,250.0,800.0,0.0,0.0,0.0,0.0,4370,12,22215.0,7899.0,322;13;159;6,9868;27100;11411,117910 44854 26564 51353 117763 57127 134820 1...


In [66]:
data = preprocess(data,dense_features) #特征处理
data = data[dense_features+sparse_features+varlen_features+['date_']+target]

In [ ]:

# def split(column):
#     if not isinstance(column,str):
#         return []
#     keys = column.strip().split(';')
#     for key in keys:
#         if key not in key2index:
#             key2index[key] = len(key2index) + 1
#     return list(map(lambda x: key2index[x], keys))


In [69]:
# 变长特征编码
# 处理manual_keyword_list和manual_tag_list变长特征
encoder = {}

def split(column):
    if not isinstance(column,str):
        return []
    keys = column.strip().split(';')
    for key in keys:
        if key not in key2index:
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], keys))


global key2index
for f in ['manual_keyword_list','manual_tag_list']:
    key2index = {}
    f_list = list(map(split, data[f].values))
    f_length = np.array(list(map(len, f_list)))
    max_len = max(f_length)
    print(f'{f} 字段最长的取值序列长度为 {max_len}')
    # Notice : padding=`post`
    data[f] = list(pad_sequences(f_list, maxlen=max_len, padding='post', ))
    encoder[f] = copy.copy(key2index)

manual_keyword_list 字段最长的取值序列长度为 18
manual_tag_list 字段最长的取值序列长度为 11


In [71]:
# 处理description变长特征
encoder_description={}

def split_by_space(column):
    if not isinstance(column,str):
        return []
    keys = column.strip().split(' ')
    for key in keys:
        if key not in key_to_index_description:
            key_to_index_description[key] = len(key_to_index_description) + 1
    return list(map(lambda x: key_to_index_description[x], keys))

global key_to_index_description
for f in ['description']:
    key_to_index_description={}
    f_list = list(map(split_by_space, data[f].values))
    f_length = np.array(list(map(len, f_list)))
    max_len = max(f_length)
    print(f'{f} 字段最长的取值序列长度为 {max_len}')
    # Notice : padding=`post`
    data[f] = list(pad_sequences(f_list, maxlen=max_len, padding='post', ))
    encoder_description[f] = copy.copy(key_to_index_description)

description 字段最长的取值序列长度为 622


In [72]:
data['description']

0          [1, 2, 3, 1, 4, 5, 6, 7, 8, 9, 10, 11, 3, 12, ...
1          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2          [39, 12, 40, 41, 42, 43, 44, 45, 20, 21, 46, 4...
3          [14, 59, 60, 61, 62, 63, 64, 65, 66, 62, 7, 67...
4          [68, 69, 19, 70, 71, 72, 35, 68, 35, 73, 74, 0...
                                 ...                        
7739862    [888, 1939, 1940, 798, 647, 1941, 1226, 353, 1...
7739863    [10, 493, 18681, 469, 7, 1549, 3587, 137, 543,...
7739864    [318, 4660, 1473, 30, 2874, 4660, 28, 9, 30, 2...
7739865    [3712, 3934, 137, 325, 13524, 7, 21, 591, 4146...
7739866    [14, 10, 194, 11659, 7, 21, 761, 20, 29, 35, 2...
Name: description, Length: 7739867, dtype: object

In [73]:
# 稀疏特征编码
for featid in sparse_features:
    print(f"编码ID字段：{featid}")
    encoder[featid] = {uid:ucode+1 for ucode,uid in enumerate(data[featid].unique())} 
    data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))

编码ID字段：userid
编码ID字段：feedid
编码ID字段：authorid
编码ID字段：bgm_song_id
编码ID字段：bgm_singer_id


In [74]:
print('数据维度：', data.shape)
print('数据字段：', data.columns.tolist())
print('不同的date_取值: ', data['date_'].unique())
# 资源比较少，可以在这里进行数据采样
data = data.sample(frac = 1.0)

数据维度： (7739867, 14)
数据字段： ['videoplayseconds', 'userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id', 'manual_tag_list', 'manual_keyword_list', 'description', 'date_', 'read_comment', 'like', 'click_avatar', 'forward']
不同的date_取值:  [ 1  2  3  5  6  7  8 10 11 12 13 14  4  9 15]


In [75]:
# 构建训练集，验证集和测试集
# 第14天样本作为验证集
train = data[data['date_'] < 14].drop(['date_'],axis = 1)
val = data[data['date_'] == 14].drop(['date_'],axis = 1)  
test = data[data['date_'] == 15].drop(['date_'],axis = 1)

In [76]:
import gc
import joblib
# gc.collect()

In [77]:
train.head()

,videoplayseconds,userid,feedid,authorid,bgm_song_id,bgm_singer_id,manual_tag_list,manual_keyword_list,description,read_comment,like,click_avatar,forward
987320,1.583341,2710,1235,776,536,483,"[34, 35, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[241, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[756, 5826, 790, 10, 7, 641, 65, 5827, 0, 0, 0...",0.0,0.0,0.0,0.0
2759866,1.310506,7563,939,615,2,2,"[54, 55, 9, 48, 33, 2, 0, 0, 0, 0, 0]","[729, 730, 626, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4416, 20, 4416, 20, 35, 4847, 270, 3758, 0, 0...",1.0,0.0,0.0,0.0
5197712,1.327761,14196,4149,2158,2,2,"[50, 11, 9, 40, 2, 0, 0, 0, 0, 0, 0]","[1967, 88, 356, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[35, 4889, 4890, 1324, 2456, 65, 2141, 2297, 1...",0.0,0.0,0.0,0.0
3883438,1.310506,10592,76182,10283,2,2,"[74, 72, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[17955, 17956, 3271, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[522, 30554, 41694, 5115, 10150, 1503, 365, 15...",1.0,0.0,0.0,0.0
3918187,1.631370,10684,5680,2797,2,2,"[44, 32, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[59, 578, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[7971, 12516, 7, 12517, 21, 3268, 8609, 7, 496...",0.0,0.0,0.0,0.0


In [78]:
joblib.dump(train, './data_and_feature/train.txt')
joblib.dump(val, './data_and_feature/val.txt')
joblib.dump(test, './data_and_feature/test.txt')
joblib.dump(encoder, './data_and_feature/encoder.txt')
joblib.dump(encoder_description, './data_and_feature/encoder_description.txt')

['./data_and_feature/encoder_description.txt']

### 生成embedding_feat_dict

In [82]:
embedding_feat_dict = generate_complete_embedding_config(data, encoder, encoder_description)

In [83]:
embedding_feat_dict

{'dense': ['videoplayseconds'],
 'sparse': {'userid': {'vocab_size': 20001, 'embedding_dim': 6},
  'feedid': {'vocab_size': 99172, 'embedding_dim': 6},
  'authorid': {'vocab_size': 18624, 'embedding_dim': 6},
  'bgm_song_id': {'vocab_size': 23740, 'embedding_dim': 6},
  'bgm_singer_id': {'vocab_size': 16603, 'embedding_dim': 6}},
 'sequence': {'manual_tag_list': {'vocab_size': 350,
   'max_len': 11,
   'embedding_dim': 6},
  'manual_keyword_list': {'vocab_size': 22145,
   'max_len': 18,
   'embedding_dim': 6},
  'description': {'vocab_size': 46489, 'max_len': 622, 'embedding_dim': 6}}}

In [84]:
save_json_file('./config/embedding_feat_dict.json', embedding_feat_dict)

文件保存成功!
